## Importing the needed libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate, GridSearchCV, StratifiedKFold
from sklearn.decomposition import PCA
from mlxtend.classifier import StackingCVClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

## Defining a weighted score

We create a weighted scorer that takes into account the class distribution obtained by the leaderboard.  
The classes more represented will have a bigger impact on the scorer, and hence the optimization will be more efficient on accuracy. 

In [2]:
# obtain test set class distribution through probing predicitons 
class_weight = {1: 0.370530,
                2: 0.496810,
                3: 0.059365,
                4: 0.001037,
                5: 0.012958,
                6: 0.026873,
                7: 0.032427}

In [3]:
def balanced_accuracy_score(y_true, y_pred):
    return accuracy_score(y_true, y_pred, sample_weight=[class_weight[label] for label in y_true])
balanced_accuracy_scorer = make_scorer(balanced_accuracy_score, greater_is_better=True)
my_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

## Loading the data ##

In [4]:
X_train = pd.read_csv("train.csv")
X_test = pd.read_csv("test-full.csv")

In [5]:
X_train.drop("Id", axis=1, inplace=True)
test_ID = X_test["Id"]
X_test.drop("Id", axis=1, inplace=True)

In [6]:
y_train = np.array(X_train['Cover_Type'])
X_train.drop('Cover_Type', axis=1, inplace=True)

In [7]:
#Check that the data is well one-hot encoded on wilderness area and 
assert np.all(X_train.loc[:, "Wilderness_Area1": "Wilderness_Area4"].sum(axis=1) == 1)
assert np.all(X_train.loc[:, "Soil_Type1": "Soil_Type40"].sum(axis=1) == 1)

In [8]:
X_train.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,2881,130,22,210,54,1020,250,221,88,342,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,3005,351,14,242,-16,1371,194,215,159,842,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3226,63,14,618,2,1092,232,210,107,2018,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,3298,317,8,661,60,752,198,233,174,1248,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3080,35,6,175,26,3705,219,227,144,2673,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Dimension reduction : PCA

In [9]:
num_train = X_train.shape[0]
all_data = pd.concat([X_train, X_test])

In [10]:
pca = PCA(n_components=0.95).fit(all_data)
pca_trans = pca.transform(all_data)
pca_trans.shape

(596132, 2)

In [11]:
for i in range(pca_trans.shape[1]):
    all_data["pca" + str(i)] = pca_trans[:, i]

## Feature Engineering ##

### Computing the distance to hydrology ###

In [12]:
all_data["Distance_to_Hydrology"] = np.sqrt(np.square(all_data["Vertical_Distance_To_Hydrology"]) +
                                     np.square(all_data["Horizontal_Distance_To_Hydrology"]))

### Mean and standard deviation of hillshade

In [13]:
hillshade_cols = ["Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm"]
all_data["Hillshade_mean"] = all_data[hillshade_cols].mean(axis=1)
all_data["Hillshade_std"] = all_data[hillshade_cols].std(axis=1)

### Combination of horizontal distances

In [14]:
cols = ["Horizontal_Distance_To_Hydrology",  "Horizontal_Distance_To_Roadways", "Horizontal_Distance_To_Fire_Points"]
names = ["H", "R", "F"]
for i in range(len(cols)):
    for j in range(i + 1, len(cols)):
        all_data["Horizontal_Distance_combination_" + names[i] + names[j] + "_1"] = all_data[cols[i]] + all_data[cols[j]]
        all_data["Horizontal_Distance_combination_" + names[i] + names[j] + "_2"] = (all_data[cols[i]] + all_data[cols[j]]) / 2
        all_data["Horizontal_Distance_combination_" + names[i] + names[j] + "_3"] = all_data[cols[i]] - all_data[cols[j]]
        all_data["Horizontal_Distance_combination_" + names[i] + names[j] + "_4"] = np.abs(all_data[cols[i]] - all_data[cols[j]])
all_data["Horizontal_Distance_mean"] = all_data[cols].mean(axis=1)

### Elevation of hydrology

In [15]:
all_data["Elevation_Hydrology_1"] = all_data["Elevation"] + all_data["Vertical_Distance_To_Hydrology"]
all_data["Elevation_Hydrology_2"] = all_data["Elevation"] - all_data["Vertical_Distance_To_Hydrology"]

In [16]:
all_data.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,pca0,pca1,Distance_to_Hydrology,Hillshade_mean,Hillshade_std,Horizontal_Distance_combination_HR_1,Horizontal_Distance_combination_HR_2,Horizontal_Distance_combination_HR_3,Horizontal_Distance_combination_HR_4,Horizontal_Distance_combination_HF_1,Horizontal_Distance_combination_HF_2,Horizontal_Distance_combination_HF_3,Horizontal_Distance_combination_HF_4,Horizontal_Distance_combination_RF_1,Horizontal_Distance_combination_RF_2,Horizontal_Distance_combination_RF_3,Horizontal_Distance_combination_RF_4,Horizontal_Distance_mean,Elevation_Hydrology_1,Elevation_Hydrology_2
0,2881,130,22,210,54,1020,250,221,88,342,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1976.101198,-688.389845,216.831732,186.333333,86.384798,1230,615.0,-810,810,552,276.0,-132,132,1362,681.0,678,678,524.000000,2935,2827
1,3005,351,14,242,-16,1371,194,215,159,842,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1407.260334,-454.764413,242.528349,189.333333,28.290163,1613,806.5,-1129,1129,1084,542.0,-600,600,2213,1106.5,529,529,818.333333,2989,3021
2,3226,63,14,618,2,1092,232,210,107,2018,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1005.892374,688.582391,618.003236,183.000000,66.730802,1710,855.0,-474,474,2636,1318.0,-1400,1400,3110,1555.0,-926,926,1242.666667,3228,3224
3,3298,317,8,661,60,752,198,233,174,1248,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1695.190318,207.353794,663.717560,201.666667,29.670412,1413,706.5,-91,91,1909,954.5,-587,587,2000,1000.0,-496,496,887.000000,3358,3238
4,3080,35,6,175,26,3705,219,227,144,2673,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1540.064230,-124.861619,176.920886,196.666667,45.785733,3880,1940.0,-3530,3530,2848,1424.0,-2498,2498,6378,3189.0,1032,1032,2184.333333,3106,3054


## Comparing models

### Defining training and test set

In [17]:
X_train = all_data[:num_train]
X_test = all_data[num_train:]

### RandomForestClassifier

In [18]:
clf_rfc = RandomForestClassifier(n_estimators=250, random_state=0, n_jobs=-1)
scores_rfc = cross_validate(clf_rfc, X_train, y_train, cv=my_cv,
                        fit_params={"sample_weight":[class_weight[label] for label in y_train]},
                        scoring=balanced_accuracy_scorer, return_train_score=True)
print(np.mean(scores_rfc["train_score"]), np.std(scores_rfc["train_score"]))
print(np.mean(scores_rfc["test_score"]), np.std(scores_rfc["test_score"]))

1.0 0.0
0.7591144495370371 0.0049230332238218485


### XGBoost

In [19]:
clf_xgb = xgb.XGBClassifier(n_estimators=50, random_state=0, n_jobs=-1)
scores_xgb = cross_validate(clf_xgb, X_train, y_train, cv=my_cv,
                        fit_params={"sample_weight":[class_weight[label] for label in y_train]},
                        scoring=balanced_accuracy_scorer, return_train_score=True)
print(np.mean(scores_xgb["train_score"]), np.std(scores_xgb["train_score"]))
print(np.mean(scores_xgb["test_score"]), np.std(scores_xgb["test_score"]))

[11:29:30] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:29:37] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:29:44] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:29:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with th

### ExtraTreesClassifier

In [20]:
clf_etc = ExtraTreesClassifier(n_estimators=250, random_state=0, n_jobs=-1)
scores_etc = cross_validate(clf_etc, X_train, y_train, cv=my_cv,
                        fit_params={"sample_weight":[class_weight[label] for label in y_train]},
                        scoring=balanced_accuracy_scorer, return_train_score=True)
print(np.mean(scores_etc["train_score"]), np.std(scores_etc["train_score"]))
print(np.mean(scores_etc["test_score"]), np.std(scores_etc["test_score"]))

1.0 0.0
0.8018592287037037 0.009538528269943518


### LGBMClassifier

In [21]:
clf_lgbmc = lgb.LGBMClassifier(n_estimators=600, random_state=0, n_jobs=-1)
scores_lgbmc = cross_validate(clf_lgbmc, X_train, y_train, cv=my_cv,
                        fit_params={"sample_weight":[class_weight[label] for label in y_train]},
                        scoring=balanced_accuracy_scorer, return_train_score=True)
print(np.mean(scores_lgbmc["train_score"]), np.std(scores_lgbmc["train_score"]))
print(np.mean(scores_lgbmc["test_score"]), np.std(scores_lgbmc["test_score"]))

1.0 0.0
0.7908917750000002 0.009332265061177264


### StackingCVClassifier

In [22]:
clf1 = ExtraTreesClassifier(n_estimators=250, random_state=0, n_jobs=-1)
clf2 = lgb.LGBMClassifier(n_estimators=600, random_state=0, n_jobs=-1)
clf = StackingCVClassifier(classifiers=[clf1, clf2],
                           meta_classifier=xgb.XGBClassifier(
                               n_estimators=50, random_state=0, n_jobs=-1),
                           cv=my_cv, random_state=0, use_probas=True, use_features_in_secondary=True)
scores_stack = cross_validate(clf, X_train, y_train, cv=my_cv,
                              fit_params={"sample_weight": [
                                  class_weight[label] for label in y_train]},
                              scoring=balanced_accuracy_scorer, return_train_score=True)
print(np.mean(scores_stack["train_score"]),
      np.std(scores_stack["train_score"]))
print(np.mean(scores_stack["test_score"]), np.std(scores_stack["test_score"]))

[11:32:15] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:33:34] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:35:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:36:38] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with th

## Predicting

In [23]:
clf.fit(X_train, y_train, sample_weight=[class_weight[label] for label in y_train])
pred = clf.predict(X_test)

[11:39:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


## Saving results

In [24]:
submission = pd.DataFrame({'Id':test_ID, 'Cover_Type':pred}, columns=['Id', 'Cover_Type'])
submission.to_csv("output.csv", index=False)